In [ ]:
# Task1: Predict total passenger flow volume for the next 5-minute time slice
# Based on cleaned CSVs:
# /mnt/data/hall_calls_clean.csv
# /mnt/data/load_changes_clean.csv
# /mnt/data/car_calls_clean.csv
# /mnt/data/car_stops_clean.csv
# /mnt/data/car_departures_clean.csv
# /mnt/data/maintenance_mode_clean.csv

import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

# -----------------------------
# 0) Paths
# -----------------------------
PATH_HALL = "/mnt/data/hall_calls_clean.csv"
PATH_LOAD = "/mnt/data/load_changes_clean.csv"
PATH_MAINT = "/mnt/data/maintenance_mode_clean.csv"

# -----------------------------
# 1) Config
# -----------------------------
FREQ = "5min"

# 客流定义方式：
# "hall_calls" = 每5分钟 hall call 数（推荐）
# "load_in"    = 每5分钟 Load In 总重量 / avg_weight 估算人数（需要你确认单位与平均体重）
FLOW_MODE = "hall_calls"

# 仅在 FLOW_MODE="load_in" 时使用：平均乘客体重（单位要与 Load In 一致）
AVG_PASSENGER_WEIGHT = 65.0  # 例如 65 kg（如果 Load In 是 kg）；不确定请改成你们合理数值

# 滞后特征：用过去多少个 5-min 片段
LAGS = [1, 2, 3, 6, 12]  # 5/10/15/30/60分钟
ROLL_WINDOWS = [3, 6, 12]  # rolling mean window size in slices

# 训练集比例（按时间顺序切分）
TRAIN_RATIO = 0.85

MODEL_OUT = "task1_flow_model.joblib"


# -----------------------------
# 2) Helpers
# -----------------------------
def add_time_features(df: pd.DataFrame, time_col="Time") -> pd.DataFrame:
    """加入能适应一天不同时段的周期特征"""
    out = df.copy()
    t = out[time_col]

    out["hour"] = t.dt.hour
    out["minute"] = t.dt.minute
    out["dow"] = t.dt.dayofweek  # Monday=0

    # 一天的分钟数 [0, 1440)
    minute_of_day = out["hour"] * 60 + out["minute"]
    # 周期编码：sin/cos（适应昼夜规律）
    out["sin_day"] = np.sin(2 * np.pi * minute_of_day / 1440.0)
    out["cos_day"] = np.cos(2 * np.pi * minute_of_day / 1440.0)

    # 一周周期（可选）
    out["sin_week"] = np.sin(2 * np.pi * out["dow"] / 7.0)
    out["cos_week"] = np.cos(2 * np.pi * out["dow"] / 7.0)

    # 是否工作日
    out["is_weekend"] = (out["dow"] >= 5).astype(int)

    return out


def add_lag_features(df: pd.DataFrame, y_col="y") -> pd.DataFrame:
    """滞后 + 滚动统计特征"""
    out = df.copy()
    for lag in LAGS:
        out[f"lag_{lag}"] = out[y_col].shift(lag)

    for w in ROLL_WINDOWS:
        out[f"roll_mean_{w}"] = out[y_col].shift(1).rolling(w).mean()
        out[f"roll_std_{w}"] = out[y_col].shift(1).rolling(w).std()

    # 简单趋势：最近1小时与前1小时差（如果有足够数据）
    if 12 in LAGS:
        out["trend_1h"] = out["lag_1"] - out["lag_12"]

    return out


def build_flow_series(freq=FREQ, mode=FLOW_MODE) -> pd.DataFrame:
    """
    构造 5分钟总客流时间序列。
    mode:
      - hall_calls: 每5min hall_calls数量
      - load_in: 每5min Load In/avg_weight 估算“进电梯人数”
    """
    if mode == "hall_calls":
        hall = pd.read_csv(PATH_HALL)
        hall["Time"] = pd.to_datetime(hall["Time"])
        # 5分钟聚合：数量
        s = hall.set_index("Time").resample(freq).size().rename("y").to_frame()

    elif mode == "load_in":
        load = pd.read_csv(PATH_LOAD)
        load["Time"] = pd.to_datetime(load["Time"])
        # 5分钟聚合：Load In 总重量 -> 估计人数
        s = load.set_index("Time")["Load In"].resample(freq).sum().fillna(0)
        s = (s / AVG_PASSENGER_WEIGHT).rename("y").to_frame()

    else:
        raise ValueError("FLOW_MODE must be 'hall_calls' or 'load_in'")

    # 补齐缺失时间片
    s = s.asfreq(freq, fill_value=0).reset_index().rename(columns={"Time": "Time"})
    return s


# -----------------------------
# 3) Train + Evaluate + Predict
# -----------------------------
def train_task1():
    # 3.1 Build target series
    df = build_flow_series()
    df["Time"] = pd.to_datetime(df["Time"])

    # 3.2 Add features
    df = add_time_features(df, "Time")
    df = add_lag_features(df, "y")

    # 3.3 Drop rows with NaNs due to lag/rolling
    df_feat = df.dropna().reset_index(drop=True)

    feature_cols = [
        "sin_day", "cos_day", "sin_week", "cos_week", "is_weekend",
        "hour", "minute",
    ] + [f"lag_{l}" for l in LAGS] + \
        [f"roll_mean_{w}" for w in ROLL_WINDOWS] + \
        [f"roll_std_{w}" for w in ROLL_WINDOWS] + (["trend_1h"] if "trend_1h" in df_feat.columns else [])

    X = df_feat[feature_cols].values
    y = df_feat["y"].values

    # 3.4 Time-based split
    n = len(df_feat)
    n_train = int(n * TRAIN_RATIO)

    X_train, y_train = X[:n_train], y[:n_train]
    X_test, y_test = X[n_train:], y[n_train:]

    # 3.5 Model
    model = HistGradientBoostingRegressor(
        loss="squared_error",
        max_depth=6,
        learning_rate=0.05,
        max_iter=400,
        random_state=42
    )
    model.fit(X_train, y_train)

    # 3.6 Evaluate
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    rmse = np.sqrt(mean_squared_error(y_test, pred))

    print("========== Task1 Result ==========")
    print(f"FLOW_MODE = {FLOW_MODE}")
    print(f"Test MAE  = {mae:.4f}")
    print(f"Test RMSE = {rmse:.4f}")

    # 3.7 Save
    payload = {
        "model": model,
        "feature_cols": feature_cols,
        "freq": FREQ,
        "flow_mode": FLOW_MODE,
        "avg_passenger_weight": AVG_PASSENGER_WEIGHT,
        "lags": LAGS,
        "roll_windows": ROLL_WINDOWS
    }
    joblib.dump(payload, MODEL_OUT)
    print(f"Saved model to: {MODEL_OUT}")

    # 3.8 Predict next 5-min slice using the latest available time slice
    next_pred, next_time = predict_next(df, payload)
    print(f"Next slice time: {next_time}")
    print(f"Predicted flow : {next_pred:.4f}")

    return payload


def predict_next(raw_df: pd.DataFrame, payload: dict):
    """
    raw_df: dataframe with columns [Time, y] at 5min freq (may include other cols)
    """
    model = payload["model"]
    feature_cols = payload["feature_cols"]

    df = raw_df.copy()
    df["Time"] = pd.to_datetime(df["Time"])

    # Rebuild features the same way
    df = add_time_features(df, "Time")
    df = add_lag_features(df, "y")
    df = df.dropna().reset_index(drop=True)

    # Take last row as "current time slice", then we predict next one by creating next time row
    last = df.iloc[-1:].copy()
    last_time = last["Time"].iloc[0]
    next_time = last_time + pd.Timedelta(FREQ)

    # Create a new row for next_time with y unknown, but lag features come from history
    next_row = pd.DataFrame({"Time": [next_time], "y": [np.nan]})
    next_row = add_time_features(next_row, "Time")

    # Build lag/rolling features for next time based on existing y series
    y_series = df.set_index("Time")["y"].copy()

    # Fill lag features
    for lag in LAGS:
        next_row[f"lag_{lag}"] = y_series.iloc[-lag] if len(y_series) >= lag else np.nan

    # Fill rolling features
    for w in ROLL_WINDOWS:
        vals = y_series.iloc[-w:] if len(y_series) >= w else y_series
        next_row[f"roll_mean_{w}"] = vals.mean() if len(vals) > 0 else np.nan
        next_row[f"roll_std_{w}"] = vals.std() if len(vals) > 1 else 0.0

    if "trend_1h" in feature_cols:
        if len(y_series) >= 12:
            next_row["trend_1h"] = y_series.iloc[-1] - y_series.iloc[-12]
        else:
            next_row["trend_1h"] = 0.0

    # Ensure all required features exist
    for c in feature_cols:
        if c not in next_row.columns:
            next_row[c] = 0.0

    X_next = next_row[feature_cols].values
    pred_next = model.predict(X_next)[0]
    return float(pred_next), str(next_time)


if __name__ == "__main__":
    train_task1()


In [ ]:
# Task2: Automatically classify current building traffic state ("modes") using real-time features
# Based on cleaned CSVs:
# /mnt/data/hall_calls_clean.csv
# /mnt/data/load_changes_clean.csv
# /mnt/data/car_calls_clean.csv
# /mnt/data/car_stops_clean.csv
# /mnt/data/car_departures_clean.csv
# /mnt/data/maintenance_mode_clean.csv

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import joblib

# -----------------------------
# 0) Paths
# -----------------------------
PATH_HALL = "/mnt/data/hall_calls_clean.csv"
PATH_CAR_CALL = "/mnt/data/car_calls_clean.csv"
PATH_CAR_STOP = "/mnt/data/car_stops_clean.csv"
PATH_CAR_DEP = "/mnt/data/car_departures_clean.csv"
PATH_LOAD = "/mnt/data/load_changes_clean.csv"
PATH_MAINT = "/mnt/data/maintenance_mode_clean.csv"

# -----------------------------
# 1) Config
# -----------------------------
FREQ = "5min"          # base slice
WINDOW = "15min"       # "real-time" window to smooth (rolling)
N_CLUSTERS = 6         # you can tune 5~8
MODEL_OUT = "task2_mode_model.joblib"

# -----------------------------
# 2) Robust column helpers
# -----------------------------
def pick_col(df, candidates, required=False):
    for c in candidates:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"None of these columns exist: {candidates}")
    return None

def normalize_direction(series):
    """
    Convert direction to +1 (Up), -1 (Down), 0 (Unknown).
    Accepts strings like 'Up','Down','U','D', 1/-1, etc.
    """
    s = series.copy()
    if pd.api.types.is_numeric_dtype(s):
        # assume up>0, down<0, else 0
        out = np.where(s > 0, 1, np.where(s < 0, -1, 0))
        return pd.Series(out, index=s.index)

    ss = s.astype(str).str.lower()
    up = ss.str.contains("up") | ss.str.fullmatch("u") | ss.str.contains("↑")
    down = ss.str.contains("down") | ss.str.fullmatch("d") | ss.str.contains("↓")
    out = np.where(up, 1, np.where(down, -1, 0))
    return pd.Series(out, index=s.index)

def floor_entropy(values):
    """Normalized entropy of floor distribution (0~1)."""
    v = pd.Series(values).dropna()
    if len(v) == 0:
        return 0.0
    counts = v.value_counts()
    p = counts / counts.sum()
    ent = -(p * np.log(p + 1e-12)).sum()
    ent_norm = ent / (np.log(len(counts) + 1e-12))
    return float(ent_norm)

# -----------------------------
# 3) Load data
# -----------------------------
def load_all():
    hall = pd.read_csv(PATH_HALL)
    car_call = pd.read_csv(PATH_CAR_CALL)
    car_stop = pd.read_csv(PATH_CAR_STOP)
    car_dep = pd.read_csv(PATH_CAR_DEP)
    load = pd.read_csv(PATH_LOAD)
    maint = pd.read_csv(PATH_MAINT)

    # parse time
    for df in [hall, car_call, car_stop, car_dep, load, maint]:
        tcol = pick_col(df, ["Time", "time", "timestamp", "Datetime", "DateTime"], required=True)
        df["Time"] = pd.to_datetime(df[tcol])
    return hall, car_call, car_stop, car_dep, load, maint

# -----------------------------
# 4) Build 5-min base features
# -----------------------------
def build_base_features(freq=FREQ):
    hall, car_call, car_stop, car_dep, load, maint = load_all()

    # --- Hall call features ---
    hall_dir_col = pick_col(hall, ["Direction", "direction", "CallDirection", "HallDirection", "UpDown", "dir"])
    hall_floor_col = pick_col(hall, ["Floor", "floor", "OriginFloor", "FromFloor", "HallFloor", "StartFloor"])

    hall_idx = hall.set_index("Time")

    hall_count = hall_idx.resample(freq).size().rename("hall_calls")

    if hall_dir_col:
        d = normalize_direction(hall_idx[hall_dir_col])
        up = (d == 1).resample(freq).sum().rename("hall_up")
        down = (d == -1).resample(freq).sum().rename("hall_down")
    else:
        up = pd.Series(dtype=float, name="hall_up")
        down = pd.Series(dtype=float, name="hall_down")

    # entropy of origin floors per slice
    if hall_floor_col:
        hall_entropy = hall_idx[hall_floor_col].resample(freq).apply(floor_entropy).rename("hall_origin_entropy")
    else:
        hall_entropy = pd.Series(index=hall_count.index, data=0.0, name="hall_origin_entropy")

    # --- Car call features (optional, inter-floor indicator) ---
    car_from_col = pick_col(car_call, ["FromFloor", "from_floor", "OriginFloor", "StartFloor", "Floor"])
    car_to_col = pick_col(car_call, ["ToFloor", "to_floor", "DestinationFloor", "DestFloor", "TargetFloor"])
    car_idx = car_call.set_index("Time")

    car_calls = car_idx.resample(freq).size().rename("car_calls")

    # fraction of "inter-floor" (from != 1 and to != 1) — if floors exist
    if car_from_col and car_to_col:
        fromf = car_idx[car_from_col]
        tof = car_idx[car_to_col]
        inter = ((fromf != 1) & (tof != 1) & (fromf.notna()) & (tof.notna()))
        inter_ratio = inter.resample(freq).mean().rename("car_inter_ratio")
    else:
        inter_ratio = pd.Series(index=car_calls.index, data=0.0, name="car_inter_ratio")

    # --- Car stops (stop intensity) ---
    stop_reason_col = pick_col(car_stop, ["Reason", "reason", "StopReason"])
    stop_dir_col = pick_col(car_stop, ["Direction", "direction", "Dir", "UpDown", "dir"])
    stop_idx = car_stop.set_index("Time")

    stops = stop_idx.resample(freq).size().rename("car_stops")

    if stop_dir_col:
        sd = normalize_direction(stop_idx[stop_dir_col])
        stops_up = (sd == 1).resample(freq).sum().rename("stops_up")
        stops_down = (sd == -1).resample(freq).sum().rename("stops_down")
    else:
        stops_up = pd.Series(dtype=float, name="stops_up")
        stops_down = pd.Series(dtype=float, name="stops_down")

    # --- Departures (service activity) ---
    dep_idx = car_dep.set_index("Time")
    deps = dep_idx.resample(freq).size().rename("departures")

    # --- Load changes (proxy for passenger movement) ---
    load_idx = load.set_index("Time")
    load_in_col = pick_col(load, ["Load In", "LoadIn", "load_in", "InLoad", "WeightIn"])
    load_out_col = pick_col(load, ["Load Out", "LoadOut", "load_out", "OutLoad", "WeightOut"])

    if load_in_col:
        load_in = load_idx[load_in_col].resample(freq).sum().fillna(0).rename("load_in_sum")
    else:
        load_in = pd.Series(index=hall_count.index, data=0.0, name="load_in_sum")

    if load_out_col:
        load_out = load_idx[load_out_col].resample(freq).sum().fillna(0).rename("load_out_sum")
    else:
        load_out = pd.Series(index=hall_count.index, data=0.0, name="load_out_sum")

    # --- Maintenance (exclude or feature) ---
    maint_idx = maint.set_index("Time")
    maint_col = pick_col(maint, ["Maintenance", "maintenance", "IsMaintenance", "MaintMode", "Mode"], required=False)
    elev_col = pick_col(maint, ["Elevator", "elevator", "CarID", "CarId", "LiftID", "ID"], required=False)

    # maintenance ratio per slice: % elevators in maintenance
    if maint_col and elev_col:
        # Expect records indicating current status; we'll treat each row as a status sample
        m = maint_idx.copy()
        m["maint_flag"] = m[maint_col].astype(int) if pd.api.types.is_numeric_dtype(m[maint_col]) else (m[maint_col].astype(str).str.lower().isin(["1","true","yes","maint","maintenance"])).astype(int)
        # within each slice, compute mean over elevators (approx)
        maint_ratio = m.groupby([pd.Grouper(freq=freq), elev_col])["maint_flag"].last().groupby(level=0).mean()
        maint_ratio = maint_ratio.reindex(hall_count.index).fillna(method="ffill").fillna(0).rename("maint_ratio")
    else:
        maint_ratio = pd.Series(index=hall_count.index, data=0.0, name="maint_ratio")

    # --- Merge all ---
    feats = pd.concat([
        hall_count,
        up.reindex(hall_count.index).fillna(0).rename("hall_up"),
        down.reindex(hall_count.index).fillna(0).rename("hall_down"),
        hall_entropy.reindex(hall_count.index).fillna(0),
        car_calls.reindex(hall_count.index).fillna(0),
        inter_ratio.reindex(hall_count.index).fillna(0),
        stops.reindex(hall_count.index).fillna(0),
        stops_up.reindex(hall_count.index).fillna(0),
        stops_down.reindex(hall_count.index).fillna(0),
        deps.reindex(hall_count.index).fillna(0),
        load_in.reindex(hall_count.index).fillna(0),
        load_out.reindex(hall_count.index).fillna(0),
        maint_ratio.reindex(hall_count.index).fillna(0),
    ], axis=1)

    feats.index.name = "Time"

    # derived ratios
    total = feats["hall_calls"].replace(0, np.nan)
    feats["up_ratio"] = (feats["hall_up"] / total).fillna(0)
    feats["down_ratio"] = (feats["hall_down"] / total).fillna(0)

    # activity score (simple)
    feats["activity"] = (
        feats["hall_calls"]
        + 0.3 * feats["car_calls"]
        + 0.5 * feats["car_stops"]
        + 0.2 * feats["departures"]
    )

    # net load (optional)
    feats["net_load"] = feats["load_in_sum"] - feats["load_out_sum"]

    # time features
    t = feats.index
    feats["hour"] = t.hour
    feats["dow"] = t.dayofweek
    minute_of_day = t.hour * 60 + t.minute
    feats["sin_day"] = np.sin(2 * np.pi * minute_of_day / 1440.0)
    feats["cos_day"] = np.cos(2 * np.pi * minute_of_day / 1440.0)

    return feats.reset_index()

# -----------------------------
# 5) Rolling window smoothing (real-time features)
# -----------------------------
def add_realtime_window(df, freq=FREQ, window=WINDOW):
    df = df.sort_values("Time").copy()
    df = df.set_index("Time")

    # choose numeric columns to smooth
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # rolling mean over window
    roll = df[num_cols].rolling(window=window, min_periods=1).mean()
    roll.columns = [f"{c}_w" for c in roll.columns]

    out = pd.concat([df, roll], axis=1).reset_index()
    return out

# -----------------------------
# 6) Cluster -> mode naming (interpretable)
# -----------------------------
def name_modes(cluster_summary: pd.DataFrame):
    """
    Given per-cluster mean features, return mapping cluster -> human-readable label.
    Uses rule-based interpretation:
      - Up-Peak: high activity + up_ratio high
      - Down-Peak: high activity + down_ratio high
      - Inter-floor: high car_inter_ratio + hall_origin_entropy high (more distributed)
      - Idle/Low: low activity
      - Balanced: medium activity, up/down balanced
    """
    mapping = {}
    # pick some columns (windowed preferred if available)
    def c(name):
        return name if name in cluster_summary.columns else f"{name}_w"

    for _, row in cluster_summary.iterrows():
        k = int(row["cluster"])
        activity = row[c("activity")]
        up = row[c("up_ratio")]
        down = row[c("down_ratio")]
        inter = row[c("car_inter_ratio")]
        ent = row[c("hall_origin_entropy")]

        if activity < cluster_summary[c("activity")].quantile(0.25):
            label = "Idle/Low"
        elif up > 0.65 and activity >= cluster_summary[c("activity")].median():
            label = "Morning Up-Peak"
        elif down > 0.65 and activity >= cluster_summary[c("activity")].median():
            label = "Evening Down-Peak"
        elif inter > 0.45 or ent > 0.70:
            label = "Meeting/Inter-floor"
        else:
            label = "Balanced/Meal-hour"

        mapping[k] = label
    return mapping

# -----------------------------
# 7) Train Task2 model
# -----------------------------
def train_task2():
    base = build_base_features()
    feat = add_realtime_window(base)

    # Use windowed features as "real-time" signals
    feature_cols = [
        "hall_calls_w",
        "up_ratio_w",
        "down_ratio_w",
        "hall_origin_entropy_w",
        "car_calls_w",
        "car_inter_ratio_w",
        "car_stops_w",
        "departures_w",
        "load_in_sum_w",
        "load_out_sum_w",
        "net_load_w",
        "maint_ratio_w",
        "sin_day", "cos_day"
    ]

    # Ensure missing columns exist (if some sources lack fields)
    for c in feature_cols:
        if c not in feat.columns:
            feat[c] = 0.0

    X = feat[feature_cols].fillna(0.0).values

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("kmeans", KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=20))
    ])
    pipe.fit(X)

    clusters = pipe.predict(X)
    feat["cluster"] = clusters

    # silhouette (rough cluster quality)
    if len(np.unique(clusters)) > 1:
        sil = silhouette_score(StandardScaler().fit_transform(X), clusters)
        print(f"Silhouette score: {sil:.4f}")
    else:
        print("Silhouette score: N/A (only one cluster)")

    # cluster summary
    summary = feat.groupby("cluster")[feature_cols].mean().reset_index()
    mapping = name_modes(summary)

    feat["mode_label"] = feat["cluster"].map(mapping)

    # Save model + config
    payload = {
        "pipeline": pipe,
        "feature_cols": feature_cols,
        "freq": FREQ,
        "window": WINDOW,
        "n_clusters": N_CLUSTERS,
        "cluster_to_mode": mapping
    }
    joblib.dump(payload, MODEL_OUT)
    print(f"Saved Task2 model to: {MODEL_OUT}")

    # show distribution
    print("\nMode counts:")
    print(feat["mode_label"].value_counts())

    # example: current mode (latest time slice)
    current = feat.iloc[-1]
    print("\nCurrent Time:", current["Time"])
    print("Current Cluster:", int(current["cluster"]))
    print("Current Mode:", current["mode_label"])

    # output a csv for report figures
    out_csv = "task2_modes_timeline.csv"
    feat[["Time", "cluster", "mode_label"] + feature_cols].to_csv(out_csv, index=False)
    print(f"Saved timeline to: {out_csv}")

    return feat, payload

# -----------------------------
# 8) Real-time classify function
# -----------------------------
def classify_current_state(latest_rows_df: pd.DataFrame, payload_path=MODEL_OUT):
    """
    If later you stream data, you can pass in a DataFrame with the same columns as base features,
    then compute window features and predict current mode.
    For now, use the saved model.
    """
    payload = joblib.load(payload_path)
    pipe = payload["pipeline"]
    feature_cols = payload["feature_cols"]
    mapping = payload["cluster_to_mode"]

    # expects latest_rows_df already has windowed columns or you can build similarly
    df = latest_rows_df.copy()
    for c in feature_cols:
        if c not in df.columns:
            df[c] = 0.0
    X = df[feature_cols].fillna(0.0).values
    cl = int(pipe.predict(X)[-1])
    return cl, mapping.get(cl, "Unknown")

if __name__ == "__main__":
    train_task2()


In [ ]:
# Task3: Dynamic Parking Strategy Model (with a lightweight, reproducible simulator)
# ------------------------------------------------------------
# Uses your cleaned logs to:
# 1) Learn demand-by-floor patterns conditioned on "building mode" (from Task2-style features)
# 2) Build a Dynamic Parking Policy (k-median on floors with predicted demand weights)
# 3) Simulate and compare 3 strategies:
#    - "last_stop": do nothing (idle stays where it is)
#    - "lobby": send idle elevators to lobby (floor=1)
#    - "dynamic": mode-aware k-median parking floors + assignment
#
# Outputs:
# - Average Waiting Time (AWT) and % Long Waits (>=60s) for each strategy
#
# Files used (cleaned CSV):
# /mnt/data/hall_calls_clean.csv
# /mnt/data/car_stops_clean.csv
# /mnt/data/car_departures_clean.csv
# /mnt/data/load_changes_clean.csv
# /mnt/data/maintenance_mode_clean.csv
# /mnt/data/car_calls_clean.csv  (optional here; not needed for basic simulation)

import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# -----------------------------
# Paths
# -----------------------------
PATH_HALL = "/mnt/data/hall_calls_clean.csv"
PATH_STOP = "/mnt/data/car_stops_clean.csv"
PATH_DEP  = "/mnt/data/car_departures_clean.csv"
PATH_LOAD = "/mnt/data/load_changes_clean.csv"
PATH_MAINT= "/mnt/data/maintenance_mode_clean.csv"

# -----------------------------
# Config (tune if you want)
# -----------------------------
N_ELEVATORS = 8
LOBBY_FLOOR = 1

# "Realistic-ish" travel parameters (you can tune)
SECONDS_PER_FLOOR = 1.5     # travel time per floor
DOOR_TIME = 8.0             # open + dwell + close
LONG_WAIT_THRESHOLD = 60.0  # seconds

# Parking decision cadence / prediction horizon
DECISION_FREQ = "5min"
PRED_HORIZON_MIN = 15       # demand for next 15 minutes to choose parking

# Cluster count for "mode" discovery
N_CLUSTERS = 6

# -----------------------------
# Robust column pickers
# -----------------------------
def pick_col(df, candidates, required=False):
    for c in candidates:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"Missing required column. Tried: {candidates}")
    return None

def normalize_direction(series):
    """Return +1 (Up), -1 (Down), 0 (Unknown)."""
    s = series
    if pd.api.types.is_numeric_dtype(s):
        return pd.Series(np.where(s > 0, 1, np.where(s < 0, -1, 0)), index=s.index)
    ss = s.astype(str).str.lower()
    up = ss.str.contains("up") | ss.str.fullmatch("u") | ss.str.contains("↑")
    down = ss.str.contains("down") | ss.str.fullmatch("d") | ss.str.contains("↓")
    return pd.Series(np.where(up, 1, np.where(down, -1, 0)), index=s.index)

def floor_entropy(values):
    v = pd.Series(values).dropna()
    if len(v) == 0:
        return 0.0
    counts = v.value_counts()
    p = counts / counts.sum()
    ent = -(p * np.log(p + 1e-12)).sum()
    return float(ent / (np.log(len(counts) + 1e-12)))

# -----------------------------
# 1) Load + standardize times
# -----------------------------
def load_data():
    hall = pd.read_csv(PATH_HALL)
    stop = pd.read_csv(PATH_STOP)
    dep  = pd.read_csv(PATH_DEP)
    load = pd.read_csv(PATH_LOAD)
    maint= pd.read_csv(PATH_MAINT)

    for df in [hall, stop, dep, load, maint]:
        tcol = pick_col(df, ["Time", "time", "timestamp", "Datetime", "DateTime"], required=True)
        df["Time"] = pd.to_datetime(df[tcol])

    return hall, stop, dep, load, maint

# -----------------------------
# 2) Build 5-min features (Task2-style) for mode clustering
# -----------------------------
def build_5min_features(freq=DECISION_FREQ):
    hall, stop, dep, load, maint = load_data()

    # Hall calls
    h_dir = pick_col(hall, ["Direction","direction","CallDirection","HallDirection","UpDown","dir"])
    h_floor = pick_col(hall, ["Floor","floor","OriginFloor","FromFloor","HallFloor","StartFloor"], required=True)
    h = hall.set_index("Time")

    hall_calls = h.resample(freq).size().rename("hall_calls")
    if h_dir:
        d = normalize_direction(h[h_dir])
        hall_up = (d == 1).resample(freq).sum().rename("hall_up")
        hall_down = (d == -1).resample(freq).sum().rename("hall_down")
    else:
        hall_up = pd.Series(index=hall_calls.index, data=0.0, name="hall_up")
        hall_down = pd.Series(index=hall_calls.index, data=0.0, name="hall_down")

    hall_ent = h[h_floor].resample(freq).apply(floor_entropy).rename("origin_entropy")

    # Stops
    s = stop.set_index("Time")
    s_floor = pick_col(stop, ["Floor","floor","StopFloor","AtFloor","CurrentFloor"])
    stops = s.resample(freq).size().rename("stops")
    if s_floor:
        # "stop entropy" can proxy spread of served floors
        stop_ent = s[s_floor].resample(freq).apply(floor_entropy).rename("stop_entropy")
    else:
        stop_ent = pd.Series(index=hall_calls.index, data=0.0, name="stop_entropy")

    # Departures
    d = dep.set_index("Time")
    departures = d.resample(freq).size().rename("departures")

    # Load changes
    ld = load.set_index("Time")
    in_col = pick_col(load, ["Load In","LoadIn","load_in","InLoad","WeightIn"])
    out_col= pick_col(load, ["Load Out","LoadOut","load_out","OutLoad","WeightOut"])
    load_in = ld[in_col].resample(freq).sum().fillna(0).rename("load_in") if in_col else pd.Series(index=hall_calls.index, data=0.0, name="load_in")
    load_out= ld[out_col].resample(freq).sum().fillna(0).rename("load_out") if out_col else pd.Series(index=hall_calls.index, data=0.0, name="load_out")

    # Maintenance ratio (rough)
    m = maint.set_index("Time")
    m_flag = pick_col(maint, ["Maintenance","maintenance","IsMaintenance","MaintMode","Mode"])
    m_eid  = pick_col(maint, ["Elevator","elevator","CarID","CarId","LiftID","ID"])
    if m_flag and m_eid:
        mm = m.copy()
        mm["maint_flag"] = (
            mm[m_flag].astype(int) if pd.api.types.is_numeric_dtype(mm[m_flag])
            else mm[m_flag].astype(str).str.lower().isin(["1","true","yes","maint","maintenance"]).astype(int)
        )
        maint_ratio = (
            mm.groupby([pd.Grouper(freq=freq), m_eid])["maint_flag"].last()
              .groupby(level=0).mean()
              .reindex(hall_calls.index).fillna(method="ffill").fillna(0)
              .rename("maint_ratio")
        )
    else:
        maint_ratio = pd.Series(index=hall_calls.index, data=0.0, name="maint_ratio")

    feats = pd.concat(
        [
            hall_calls,
            hall_up.reindex(hall_calls.index).fillna(0),
            hall_down.reindex(hall_calls.index).fillna(0),
            hall_ent.reindex(hall_calls.index).fillna(0),
            stops.reindex(hall_calls.index).fillna(0),
            stop_ent.reindex(hall_calls.index).fillna(0),
            departures.reindex(hall_calls.index).fillna(0),
            load_in.reindex(hall_calls.index).fillna(0),
            load_out.reindex(hall_calls.index).fillna(0),
            maint_ratio.reindex(hall_calls.index).fillna(0),
        ],
        axis=1
    )

    total = feats["hall_calls"].replace(0, np.nan)
    feats["up_ratio"] = (feats["hall_up"] / total).fillna(0)
    feats["down_ratio"] = (feats["hall_down"] / total).fillna(0)

    # time cyc features
    t = feats.index
    minute_of_day = t.hour * 60 + t.minute
    feats["sin_day"] = np.sin(2*np.pi*minute_of_day/1440.0)
    feats["cos_day"] = np.cos(2*np.pi*minute_of_day/1440.0)

    feats["activity"] = feats["hall_calls"] + 0.5*feats["stops"] + 0.2*feats["departures"]

    feats = feats.reset_index().rename(columns={"index":"Time"})
    feats["Time"] = pd.to_datetime(feats["Time"])
    return feats

def train_mode_cluster(feats: pd.DataFrame, n_clusters=N_CLUSTERS):
    feature_cols = [
        "hall_calls","up_ratio","down_ratio","origin_entropy",
        "stops","stop_entropy","departures",
        "load_in","load_out","maint_ratio",
        "sin_day","cos_day","activity"
    ]
    X = feats[feature_cols].fillna(0.0).values
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("kmeans", KMeans(n_clusters=n_clusters, random_state=42, n_init=20))
    ])
    pipe.fit(X)
    feats = feats.copy()
    feats["cluster"] = pipe.predict(X)
    return pipe, feature_cols, feats

def label_clusters(feats_with_cluster: pd.DataFrame) -> Dict[int, str]:
    """Map cluster -> human-readable mode label (simple explainable rules)."""
    g = feats_with_cluster.groupby("cluster").mean(numeric_only=True)

    mapping = {}
    act_q25 = g["activity"].quantile(0.25)
    act_med = g["activity"].median()

    for k, row in g.iterrows():
        activity = row["activity"]
        up = row["up_ratio"]
        down = row["down_ratio"]
        ent = row["origin_entropy"]
        stop_ent = row["stop_entropy"]

        if activity <= act_q25:
            label = "Idle/Low"
        elif up > 0.65 and activity >= act_med:
            label = "Up-Peak"
        elif down > 0.65 and activity >= act_med:
            label = "Down-Peak"
        elif ent > 0.70 or stop_ent > 0.70:
            label = "Inter-floor/Meeting"
        else:
            label = "Balanced/Meal-hour"
        mapping[int(k)] = label

    return mapping

# -----------------------------
# 3) Learn demand weights by (mode, floor)
# -----------------------------
def learn_floor_demand_by_mode(hall: pd.DataFrame, feats_clustered: pd.DataFrame, freq=DECISION_FREQ):
    """
    Returns:
      demand[(mode_label)][floor] = avg hall-call count per horizon window
    """
    # Identify hall origin floor column
    h_floor = pick_col(hall, ["Floor","floor","OriginFloor","FromFloor","HallFloor","StartFloor"], required=True)

    # Attach cluster/mode to each hall call by matching its 5-min slice
    hall2 = hall.copy()
    hall2["slice"] = hall2["Time"].dt.floor(freq)

    cluster_map = feats_clustered.set_index("Time")["cluster"].to_dict()
    hall2["cluster"] = hall2["slice"].map(cluster_map)

    # Drop calls outside feature range
    hall2 = hall2.dropna(subset=["cluster"])
    hall2["cluster"] = hall2["cluster"].astype(int)

    # Count hall calls per (slice, cluster, floor)
    counts = (
        hall2.groupby(["slice","cluster", hall2[h_floor]])["Time"]
             .size()
             .rename("cnt")
             .reset_index()
             .rename(columns={h_floor:"floor"})
    )

    # Average demand per slice for each cluster-floor
    avg = counts.groupby(["cluster","floor"])["cnt"].mean().reset_index()

    return avg  # columns: cluster, floor, cnt

# -----------------------------
# 4) k-median on 1D floors (choose parking floors for k idle elevators)
# -----------------------------
def weighted_k_median_1d(floors: np.ndarray, weights: np.ndarray, k: int) -> List[int]:
    """
    Choose k facility locations on 1D line to minimize sum_i w_i * |x_i - facility(x_i)|.
    Facilities must be at existing floor positions (integers).
    DP solution O(k*n^2) for n unique floors, small n => OK.
    """
    # Sort
    order = np.argsort(floors)
    x = floors[order].astype(int)
    w = weights[order].astype(float)
    n = len(x)

    # Precompute cost[i][j] = cost of serving points i..j with 1 median facility
    # Median index m is weighted median. For simplicity (n small), brute find best median.
    cost = np.zeros((n, n))
    best_m = np.zeros((n, n), dtype=int)

    for i in range(n):
        for j in range(i, n):
            idx = np.arange(i, j+1)
            # find best median among idx
            best = None
            best_idx = i
            for m in idx:
                c = np.sum(w[idx] * np.abs(x[idx] - x[m]))
                if best is None or c < best:
                    best = c
                    best_idx = m
            cost[i, j] = best if best is not None else 0.0
            best_m[i, j] = best_idx

    # DP: dp[t][j] = min cost using t facilities to cover points 0..j
    dp = np.full((k+1, n), np.inf)
    prev = np.full((k+1, n), -1, dtype=int)

    # base: 1 facility
    for j in range(n):
        dp[1, j] = cost[0, j]
        prev[1, j] = -1

    for t in range(2, k+1):
        for j in range(n):
            # split at p: cover 0..p with t-1, and p+1..j with 1
            for p in range(t-2, j):
                cand = dp[t-1, p] + cost[p+1, j]
                if cand < dp[t, j]:
                    dp[t, j] = cand
                    prev[t, j] = p

    # Recover segments
    facilities = []
    t = k
    j = n-1
    while t >= 1 and j >= 0:
        p = prev[t, j]
        i = 0 if p == -1 else p+1
        m = best_m[i, j]
        facilities.append(int(x[m]))
        j = p
        t -= 1

    facilities.reverse()
    return facilities

def assign_elevators_to_targets(elevator_floors: Dict[int,int], targets: List[int]) -> Dict[int,int]:
    """
    Greedy matching: assign each target to closest available elevator.
    Returns: elevator_id -> target_floor
    """
    eids = list(elevator_floors.keys())
    remaining = set(eids)
    assignment = {}

    for tgt in targets:
        best_e, best_d = None, None
        for e in list(remaining):
            d = abs(elevator_floors[e] - tgt)
            if best_d is None or d < best_d:
                best_d = d
                best_e = e
        if best_e is None:
            break
        assignment[best_e] = tgt
        remaining.remove(best_e)

    # Any leftover elevators (if more idle than targets): keep them where they are
    for e in remaining:
        assignment[e] = elevator_floors[e]
    return assignment

# -----------------------------
# 5) Lightweight event simulator for waiting time
# -----------------------------
@dataclass
class ElevatorState:
    floor: int
    available_time: pd.Timestamp  # when elevator can take new call

def travel_time_seconds(f1: int, f2: int) -> float:
    return abs(int(f1) - int(f2)) * SECONDS_PER_FLOOR

def infer_initial_positions(stop: pd.DataFrame, start_time: pd.Timestamp) -> Dict[int,int]:
    """
    Infer elevator last known floor before start_time from car_stops.
    Falls back to lobby if unknown.
    """
    eid_col = pick_col(stop, ["Elevator","elevator","CarID","CarId","LiftID","ID"], required=False)
    floor_col = pick_col(stop, ["Floor","floor","StopFloor","AtFloor","CurrentFloor"], required=False)

    pos = {i: LOBBY_FLOOR for i in range(N_ELEVATORS)}
    if eid_col is None or floor_col is None:
        return pos

    s = stop[stop["Time"] <= start_time].copy()
    if s.empty:
        return pos

    s = s.sort_values("Time")
    last = s.groupby(eid_col).tail(1)
    for _, r in last.iterrows():
        try:
            eid = int(r[eid_col])
            pos[eid] = int(r[floor_col])
        except Exception:
            pass
    return pos

def simulate(strategy: str,
             hall: pd.DataFrame,
             feats: pd.DataFrame,
             mode_pipe: Pipeline,
             mode_features: List[str],
             cluster_to_label: Dict[int,str],
             demand_by_cluster_floor: pd.DataFrame,
             start_time: Optional[pd.Timestamp]=None,
             end_time: Optional[pd.Timestamp]=None) -> Dict[str,float]:
    """
    strategy in {"last_stop","lobby","dynamic"}.
    """
    # columns
    h_floor = pick_col(hall, ["Floor","floor","OriginFloor","FromFloor","HallFloor","StartFloor"], required=True)

    hall = hall.sort_values("Time").copy()
    if start_time is None:
        start_time = hall["Time"].min()
    if end_time is None:
        end_time = hall["Time"].max()

    # Filter simulation interval
    hall_sim = hall[(hall["Time"] >= start_time) & (hall["Time"] <= end_time)].copy()
    if hall_sim.empty:
        return {"AWT": np.nan, "LongWaitPct": np.nan, "N": 0}

    # Initial elevator states
    # (use last stop before start_time if possible; otherwise lobby)
    _, stop, _, _, _ = load_data()
    init_pos = infer_initial_positions(stop, start_time)

    elevators = {
        e: ElevatorState(floor=init_pos.get(e, LOBBY_FLOOR), available_time=start_time)
        for e in range(N_ELEVATORS)
    }

    # Precompute demand lookup: (cluster -> arrays floors, weights)
    demand_group = demand_by_cluster_floor.groupby("cluster")
    demand_lookup = {}
    for cl, dfc in demand_group:
        floors = dfc["floor"].astype(int).values
        w = dfc["cnt"].astype(float).values
        # avoid all-zero
        if w.sum() <= 0:
            w = np.ones_like(w)
        demand_lookup[int(cl)] = (floors, w)

    waits = []

    # Decision times for parking
    decision_times = pd.date_range(start=start_time.floor(DECISION_FREQ),
                                   end=end_time.ceil(DECISION_FREQ),
                                   freq=DECISION_FREQ)

    # Helper to get cluster at a decision time (nearest available feature row)
    feats_idx = feats.set_index("Time").sort_index()

    def get_cluster_label_at(t: pd.Timestamp) -> Tuple[int,str]:
        # use exact row if exists, else previous
        tt = t.floor(DECISION_FREQ)
        if tt in feats_idx.index:
            row = feats_idx.loc[tt]
        else:
            # previous available
            row = feats_idx.loc[:tt].tail(1)
            if isinstance(row, pd.DataFrame):
                row = row.iloc[0]
        X = row[mode_features].fillna(0.0).values.reshape(1, -1)
        cl = int(mode_pipe.predict(X)[0])
        return cl, cluster_to_label.get(cl, "Unknown")

    # Parking policy invoked at decision times
    def apply_parking_policy(t: pd.Timestamp):
        # Identify idle elevators (available_time <= t)
        idle = [e for e, st in elevators.items() if st.available_time <= t]

        if len(idle) == 0:
            return

        if strategy == "last_stop":
            return

        if strategy == "lobby":
            for e in idle:
                st = elevators[e]
                if st.floor != LOBBY_FLOOR:
                    tt = travel_time_seconds(st.floor, LOBBY_FLOOR)
                    st.available_time = t + pd.Timedelta(seconds=tt)
                    st.floor = LOBBY_FLOOR
            return

        if strategy == "dynamic":
            cl, _ = get_cluster_label_at(t)

            # demand for next horizon: scale weights by horizon slices
            if cl in demand_lookup:
                floors, w = demand_lookup[cl]
            else:
                # fallback to global
                allf = demand_by_cluster_floor.groupby("floor")["cnt"].mean().reset_index()
                floors = allf["floor"].astype(int).values
                w = allf["cnt"].astype(float).values
                if w.sum() <= 0:
                    w = np.ones_like(w)

            # Choose k target parking floors for number of idle elevators
            k = min(len(idle), N_ELEVATORS)
            # Reduce to unique floors if too many
            if len(floors) == 0:
                targets = [LOBBY_FLOOR] * k
            else:
                # If floors extremely many, keep top by weight to stabilize DP
                if len(floors) > 60:
                    top_idx = np.argsort(-w)[:60]
                    floors2, w2 = floors[top_idx], w[top_idx]
                else:
                    floors2, w2 = floors, w

                # normalize weights
                w2 = w2 / (w2.sum() + 1e-12)

                # Compute k-median targets
                targets = weighted_k_median_1d(floors2, w2, k)

            # Assign idle elevators to targets
            cur_pos = {e: elevators[e].floor for e in idle}
            assignment = assign_elevators_to_targets(cur_pos, targets)

            for e, tgt in assignment.items():
                st = elevators[e]
                if st.floor != tgt:
                    tt = travel_time_seconds(st.floor, tgt)
                    st.available_time = t + pd.Timedelta(seconds=tt)
                    st.floor = tgt

    # Simulation loop over calls; we also apply parking at decision points
    decision_ptr = 0
    decision_times = list(decision_times)

    for _, call in hall_sim.iterrows():
        call_time = call["Time"]
        origin_floor = int(call[h_floor])

        # Apply parking decisions up to this call_time
        while decision_ptr < len(decision_times) and decision_times[decision_ptr] <= call_time:
            apply_parking_policy(decision_times[decision_ptr])
            decision_ptr += 1

        # Choose an elevator to serve this call:
        # minimize arrival time = max(available_time, call_time) + travel_time(current_floor -> origin)
        best_e = None
        best_arrival = None

        for e, st in elevators.items():
            start_service = max(st.available_time, call_time)
            arr = start_service + pd.Timedelta(seconds=travel_time_seconds(st.floor, origin_floor))
            if best_arrival is None or arr < best_arrival:
                best_arrival = arr
                best_e = e

        # Waiting time: arrival - call_time (doors open assumed at arrival + DOOR_TIME doesn't affect waiting-to-arrival)
        wait_sec = (best_arrival - call_time).total_seconds()
        waits.append(wait_sec)

        # Update chosen elevator state: after serving, elevator stays at origin floor, becomes available after door time
        st = elevators[best_e]
        st.floor = origin_floor
        st.available_time = best_arrival + pd.Timedelta(seconds=DOOR_TIME)

    waits = np.array(waits, dtype=float)
    awt = float(np.mean(waits)) if len(waits) else np.nan
    long_pct = float(np.mean(waits >= LONG_WAIT_THRESHOLD) * 100.0) if len(waits) else np.nan

    return {"AWT": awt, "LongWaitPct": long_pct, "N": int(len(waits))}

# -----------------------------
# 6) Train everything + run comparison
# -----------------------------
def run_task3_demo():
    hall, _, _, _, _ = load_data()

    # (A) Build mode features + cluster model
    feats = build_5min_features()
    mode_pipe, mode_features, feats_clustered = train_mode_cluster(feats, n_clusters=N_CLUSTERS)
    cl_to_label = label_clusters(feats_clustered)

    # (B) Learn demand by (cluster, floor)
    demand = learn_floor_demand_by_mode(hall, feats_clustered)

    # (C) Choose a simulation window (e.g., last 3 days) to keep runtime reasonable
    # You can expand to full 30 days once happy.
    tmin = hall["Time"].min()
    tmax = hall["Time"].max()
    sim_start = max(tmin, tmax - pd.Timedelta(days=3))
    sim_end = tmax

    # (D) Evaluate strategies
    results = {}
    for strat in ["last_stop", "lobby", "dynamic"]:
        res = simulate(
            strategy=strat,
            hall=hall,
            feats=feats_clustered,
            mode_pipe=mode_pipe,
            mode_features=mode_features,
            cluster_to_label=cl_to_label,
            demand_by_cluster_floor=demand,
            start_time=sim_start,
            end_time=sim_end
        )
        results[strat] = res

    print("\n========== Task3 Strategy Comparison ==========")
    print(f"Simulation window: {sim_start}  ->  {sim_end}")
    for k, v in results.items():
        print(f"{k:>10} | AWT={v['AWT']:.2f}s | LongWait%={v['LongWaitPct']:.2f}% | N={v['N']}")

    # Save artifacts useful for report
    feats_clustered.to_csv("task3_mode_features_5min.csv", index=False)
    demand.to_csv("task3_demand_cluster_floor.csv", index=False)

    return results

if __name__ == "__main__":
    run_task3_demo()
